In [1]:
import praw
import pandas as pd

In [2]:
# Required to use the PRAW api and setting up an instance
client_id = 'EbwL3_aj_O6n_A'
client_sec = 'yMO-6kRawzDO7uCrnE6-ssLzJh0'
user_agt = 'RedditWebScraping'
reddit = praw.Reddit(client_id=client_id, 
                     client_secret=client_sec, 
                     user_agent=user_agt)

In [3]:
# Setting up the subreddits
wsb = 'wallstreetbets'
btc = 'bitcoin'
wsb_sub = reddit.subreddit(wsb)
btc_sub = reddit.subreddit(btc)


In [24]:
# Getting the posts from a subreddit, will then add it to a list
def get_posts(sub):
    output = []
    for post in sub.top(limit=2_500):
        output.append(subreddit_dict(post))
    return output

In [11]:
# Converting from PRAW-class data to useable data and setting it 
# up for use in a Pandas dataframe
def subreddit_dict(post):
    d = {
        'title' : post.title,
        'id' : post.id,
        'url' : post.url,
        'body' : post.selftext,
        'date_created' : post.created,
        'number_comments' : post.num_comments,
        'author' : post.author
    }
    return d

In [12]:
# Getting all the comments from a post and putting it into a list
def get_comments(post):
    submission = reddit.submission(id=post)
    cmts = []
    submission.comments.replace_more(limit=0)
    for comment in submission.comments:
        cmts.append({
            'id' : post,
            'comment' : comment.body
        })
    return cmts

In [13]:
# Getting all the comments from the posts I scraped from a subreddit
def all_comments(subreddit):
    all_com = []
    for post in subreddit:
        coms = get_comments(subreddit['id'])
        for comment in coms:
            all_com.append(comment)
    return all_com

In [14]:
# Exporting a dataframe as a csv
def export_csv(data, name):
    df = pd.DataFrame(data)
    df.to_csv(f'./data/{name}.csv', index=False)

In [25]:
# This is the code I used to get all the posts and comments from a subreddit
# Also, I set this to convert it to a Dataframe and then export it
%%time
wsb_posts = get_posts(wsb_sub)
wsb_com = []
for post in wsb_posts:
    wsb_com.append(all_comments(post))
btc_posts = get_posts(btc_sub)
btc_com = []
for post in btc_posts:
    btc_com.append(all_comments(post))
    
export_csv(wsb_posts, 'wsb_posts')
# export_csv(wsb_com, 'wsb_comments')
export_csv(btc_posts, 'btc_posts')
# export_csv(btc_com, 'btc_comments')

CPU times: user 10min 24s, sys: 10 s, total: 10min 34s
Wall time: 6h 4min 13s


In [34]:
# The comments were formatted incorrectly, this corrected it
def comment_df(coms):
    comments = []
    for post in coms:
        for com in post:
            comments.append(com)
    return comments
    

In [35]:
export_csv(comment_df(wsb_com), 'wsb_comments')
export_csv(comment_df(btc_com), 'btc_comments')